In [22]:
import numpy as np
import pandas as pd
import warnings
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MaxAbsScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import classification_report

# Set a random seed
random_seed = 42
np.random.seed(random_seed)
warnings.filterwarnings('ignore')

Data Collection and Analysis

PIMA Diabetes Dataset

In [23]:
# loading the diabetes dataset to a pands DataFrame
diabetes_data = pd.read_csv('diabetes.csv')

In [24]:
# printing the first 5 rows of the dataset
diabetes_data.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [25]:
diabetes_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [26]:
diabetes_data.describe().T

,count,mean,std,min,25%,50%,75%,max
Pregnancies,768.0,3.845052,3.369578,0.000,1.00000,3.0000,6.00000,17.00
Glucose,768.0,120.894531,31.972618,0.000,99.00000,117.0000,140.25000,199.00
BloodPressure,768.0,69.105469,19.355807,0.000,62.00000,72.0000,80.00000,122.00
SkinThickness,768.0,20.536458,15.952218,0.000,0.00000,23.0000,32.00000,99.00
Insulin,768.0,79.799479,115.244002,0.000,0.00000,30.5000,127.25000,846.00
BMI,768.0,31.992578,7.884160,0.000,27.30000,32.0000,36.60000,67.10
DiabetesPedigreeFunction,768.0,0.471876,0.331329,0.078,0.24375,0.3725,0.62625,2.42
Age,768.0,33.240885,11.760232,21.000,24.00000,29.0000,41.00000,81.00
Outcome,768.0,0.348958,0.476951,0.000,0.00000,0.0000,1.00000,1.00


In [27]:
diabetes_data.columns = ['pregnancies','glucose','blood_pressure','skin_thickness','insulin','bmi','diabetes_pedigree_function','age','outcome']

0 --> Non-Diabetic

1 --> Diabetic

In [28]:
diabetes_data['outcome'].value_counts()

outcome
0    500
1    268
Name: count, dtype: int64

In [29]:
# separating the data and labels
X = diabetes_data.drop(columns=['outcome'])
Y = diabetes_data['outcome']

In [30]:
# Random oversamplingrandom_state=random_seed
oversampler = RandomOverSampler(random_state=random_seed)
X_resampled, Y_resampled = oversampler.fit_resample(X,Y)

In [31]:
Y_resampled.value_counts()

outcome
1    500
0    500
Name: count, dtype: int64

Train Test Split

In [32]:
x_train, x_test, y_train, y_test = train_test_split(X_resampled,Y_resampled,test_size=0.2,random_state=random_seed)

In [33]:
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

(800, 8) (200, 8) (800,) (200,)


Training the Model

In [34]:
model_pipeline = Pipeline([
    ('scaler',MaxAbsScaler()),
    ('classifier',ExtraTreesClassifier())
])
model_pipeline.fit(x_train,y_train)

Pipeline(steps=[('scaler', MaxAbsScaler()),
                ('classifier', ExtraTreesClassifier())])

In [35]:
y_pred = model_pipeline.predict(x_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.85      0.82      0.84        99
           1       0.83      0.86      0.84       101

    accuracy                           0.84       200
   macro avg       0.84      0.84      0.84       200
weighted avg       0.84      0.84      0.84       200



Making a Predictive System

In [36]:
input_data = (5,166,72,19,175,25.8,0.587,51)

# changing the input_data to numpy array
input_data_as_numpy_array = np.asarray(input_data)

# reshape the array as we are predicting for one instance
input_data_reshaped = input_data_as_numpy_array.reshape(1,-1)

prediction = model_pipeline.predict(input_data_reshaped)
print(prediction)

if (prediction[0] == 0):
  print('The person is not diabetic')
else:
  print('The person is diabetic')

[1]
The person is diabetic


Saving the trained model

In [37]:
import pickle

In [38]:
filename = 'diabetes-prediction-rfc-model'
pickle.dump(model_pipeline, open(filename, 'wb'))

In [39]:
# loading the saved model
loaded_model = pickle.load(open('diabetes-prediction-rfc-model', 'rb'))

In [40]:
input_data = (5,166,72,19,175,25.8,0.587,51)

# changing the input_data to numpy array
input_data_as_numpy_array = np.asarray(input_data)

# reshape the array as we are predicting for one instance
input_data_reshaped = input_data_as_numpy_array.reshape(1,-1)

prediction = loaded_model.predict(input_data_reshaped)
print(prediction)

if (prediction[0] == 0):
  print('The person is not diabetic')
else:
  print('The person is diabetic')

[1]
The person is diabetic
